# 常见应用场景

In [11]:
# 导入DeepSeek API KEY
import os
#os.environ["DEEPSEEK_API_KEY"]='<YOUR_DEEPSEEK_API_KEY>'
os.environ["DEEPSEEK_API_KEY"]='sk-c62321baee094186834df252bc531fff'
os.environ["LLM_MODEL"]='deepseek-chat'

## 意图识别

In [12]:
from openai import OpenAI
import os

client = OpenAI(
    api_key=os.getenv("DEEPSEEK_API_KEY"),
    base_url="https://api.deepseek.com")

In [13]:
import openai
response = client.chat.completions.create(
    model=os.getenv("LLM_MODEL"),
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input 
         """},
        #{"role": "user", "content": "订明天早5点北京到上海的飞机"}
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

好的，我会提醒你明早8点有会议。请确保你的设备已开启提醒功能。如果有其他需要帮助的地方，请随时告诉我！


## 生成SQL

安装langchain

In [15]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ langchain

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install -i https://mirrors.aliyun.com/pypi/simple/ langchain_community

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.0 MB/s eta 0:00:00MB/s eta 0:00:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can anwser the user request based on the given tables:
                  table “students“ with the columns [id, name, course_id, score] 
                  table "courses" with the columns [id, name] 
                  """

prompt = system_prompt

response = client.chat.completions.create(
    model=os.getenv("LLM_MODEL"),
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "计算所有学生英语课程的平均成绩"},
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

要计算所有学生英语课程的平均成绩，我们需要执行以下步骤：

1. 首先，我们需要从 `courses` 表中找到英语课程的 `id`。
2. 然后，我们从 `students` 表中筛选出 `course_id` 等于英语课程 `id` 的记录。
3. 最后，我们计算这些记录的 `score` 的平均值。

假设英语课程的名称是 "English"，我们可以使用以下 SQL 查询来实现：

```sql
SELECT AVG(s.score) AS average_score
FROM students s
JOIN courses c ON s.course_id = c.id
WHERE c.name = 'English';
```

这个查询的解释如下：

- `JOIN courses c ON s.course_id = c.id`：将 `students` 表和 `courses` 表连接起来，以便我们可以根据课程名称进行筛选。
- `WHERE c.name = 'English'`：筛选出课程名称为 "English" 的记录。
- `AVG(s.score)`：计算筛选出的记录的 `score` 的平均值。

执行这个查询后，你将得到所有学生英语课程的平均成绩。


从上面的例子可以看出，虽然AI能够生成对应的回答，但是生成的回答无法直接对接到程序。

## 生成代码友好的提示词

### 规范输出的格式

In [18]:
import openai
response = client.chat.completions.create(
    model=os.getenv("LLM_MODEL"),
    temperature = 0,
    messages=[
        {"role": "system", "content": """
          Recognize the intent from the user's input and format output as JSON string. 
        The output JSON string includes: "intention", "paramters" """},
        {"role": "user", "content": "提醒我明早8点有会议"}
    ]
  )
print(response.choices[0].message.content)

```json
{
  "intention": "set_reminder",
  "parameters": {
    "time": "08:00",
    "event": "会议",
    "date": "明天"
  }
}
```


In [20]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

prompt = system_prompt

response = client.chat.completions.create(
    model=os.getenv("LLM_MODEL"),
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        #{"role": "user", "content": "列出英语成绩大于80分的学生"},
        {"role": "user", "content": "列出英语课程成绩大于80分的学生, 返回结果只包括SQL"},
        #{"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

```sql
SELECT s.id, s.name, s.score
FROM students s
JOIN courses c ON s.course_id = c.id
WHERE c.name = 'English' AND s.score > 80;
```


### 文本规范异常输出的格式 

In [23]:
import openai, os
from langchain.llms import OpenAI

system_prompt =  """  You are a software engineer, you can write a SQL string as the anwser according to the user request 
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

system_prompt_with_negative =  """  
You are a software engineer, you can write a SQL string as the anwser according to the user request.
Also, when you cannot create the SQL query for the user's request based on the given tables, please, only return "invalid request"
               The user's requirement is based on the given tables:
                  table “students“ with the columns [id, name, course_id, score];
                  table "courses" with the columns [id, name]."""

#prompt = system_prompt
prompt = system_prompt_with_negative

response = client.chat.completions.create(
    model=os.getenv("LLM_MODEL"),
    temperature = 0,
    messages=[
        {"role": "system", "content": prompt},
        {"role": "user", "content": "列出年龄大于13的学生"}
    ],
    max_tokens = 500
  )
print(response.choices[0].message.content)

根据提供的表结构，没有包含“年龄”这一列，因此无法根据年龄来筛选学生。所以，这是一个无效的请求。

```sql
invalid request
```
